<h1> ハイパーパラメータ チューニング</h1>

このnotebookでは、ハイパーパラメータ チューニングのやり方を学んでいきましょう。

すべて実行するには数時間かかることがあります。

<h2> プロジェクトとバケットを環境変数を入力する</h2>

以下に注意してください:
<ol>
<li> Project IDは使用するプロジェクトを表す*ユニーク*な文字列です（プロジェクト名とは異なります）。GCPコンソールのホーム画面から確認することができます。 </li>
<li> クラウド上での機械学習のトレーニングでは、モデルファイルを保存したり復元したりというプロセスを含んでいます。まだバケットを持っていない場合、GCPコンソールからバケットを一つ作成してください。Project IDを先頭につけることでユニークなバケット名をつけることが一般的です。またコスト上の理由から、単一のRegionでバケットを作成するのがよいでしょう。</li>
</ol>
自分のProject IDとバケット名に<b>以下のセルの変数を変更してください。</b>

In [ ]:
import os
PROJECT = 'cloud-training-demos' # ご自分のProject IDに変更してください。
BUCKET = 'cloud-training-demos-ml' # ご自分のバケット名に変更してください。選択したリージョンのリージョナルなバケットを使用してください。
REGION = 'us-central1' # Cloud MLEを利用できるリージョンを選択してください https://cloud.google.com/ml-engine/docs/regions

In [ ]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

<h1> 1. コマンドラインからtask.pyにパラメータを渡す </h1>

コマンドラインからtask.pyに渡すパラメータに注目してください。必要に応じて、これらに対してハイパーパラメータ チューニングを行うことができます。

In [ ]:
!grep -A 2 add_argument taxifare/trainer/task.py

<h1> 2. 評価指標</h1>

独自の評価指標を加えることができます。どのような関数でも構いません。

In [ ]:
!grep -A 5 add_eval_metrics taxifare/trainer/model.py

<h1> 3. 出力が互いに衝突しないことを確認する </h1>

出力先のディレクトリにイテレーションの番号を入れてファイル名を区別しましょう

In [ ]:
!grep -A 5 "trial" taxifare/trainer/task.py

<h1> 4. ハイパーパラメータの設定を作成する </h1>

このファイルはパラメータ空間の探索領域を定義します。Cloud MLEはこの制約の中で賢く領域の探索を行います。（つまり、すべての値を試すわけではありません）

In [ ]:
%%writefile hyperparam.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 30
    maxParallelTrials: 8
    hyperparameterMetricTag: rmse
    params:
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 64
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: nbuckets
      type: INTEGER
      minValue: 10
      maxValue: 20
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: hidden_units
      type: CATEGORICAL
      categoricalValues: ["128 32", "256 128 16", "64 64 64 8"]       

<h1> 5. 学習ジョブを実行する</h1>

--configにファイルを指定し、その他は通常通りに学習コマンドを実行します。

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/ch4_hyp/taxi_trained
JOBNAME=lab4a_hyp_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version=1.4 \
   --config=hyperparam.yaml \
   -- \
   --train_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/train*" \
   --eval_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=5000

<h2>6. 選ばれたモデルを使ってデータセット全件で学習を行う</h2>

<a href="feateng.ipynb">特徴量エンジニアリングnotebook</a>の最後のタスクを確認してください。パラメータはハイパーパラメータ・チューニングの結果を利用します。

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License